In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import folium
from folium.plugins import HeatMap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# Пути
excel_path = '/content/drive/MyDrive/Г/predict/crimes_spb.xlsx'
output_csv = '/content/drive/MyDrive/Г/predict/crimes_spb_features.csv'
risk_map_path = '/content/drive/MyDrive/Г/predict/risk_map_next_3h.html'

# 1. Загрузка данных
print("🚀 Загрузка данных...")
df = pd.read_excel(excel_path)
df['Время регистрации'] = pd.to_datetime(df['Время регистрации'], errors='coerce')
df = df.dropna(subset=['Время регистрации'])

# 2. Извлечение признаков
print("⚙️ Извлечение признаков...")
df['Час'] = df['Время регистрации'].dt.hour
df['День_недели'] = df['Время регистрации'].dt.dayofweek
df['Месяц'] = df['Время регистрации'].dt.month
df['Год'] = df['Время регистрации'].dt.year
df['Время_суток'] = pd.cut(df['Час'], bins=[-0.1, 6, 12, 18, 24], labels=[0, 1, 2, 3])  # ночь, утро, день, вечер

# Целевая переменная
df['Стрельба'] = (
    df['Причина'].str.contains('стрельба|огнестрел|выстрел', case=False, na=False) |
    df['Причина АПК БГ'].str.contains('стрельба|огнестрел|выстрел', case=False, na=False)
).astype(int)

# 3. Историческая активность (за 90 дней)
print("📊 Расчёт истории стрельбы...")
def rolling_firearms(group):
    group = group.sort_values('Время регистрации').reset_index(drop=True)
    group['История_90дн'] = 0
    for i in range(len(group)):
        current_time = group.iloc[i]['Время регистрации']
        cutoff = current_time - pd.Timedelta(days=90)
        count = group[
            (group['Время регистрации'] < current_time) &
            (group['Время регистрации'] >= cutoff)
        ]['Стрельба'].sum()
        group.loc[i, 'История_90дн'] = count
    return group

df = df.groupby('Район', group_keys=False).apply(rolling_firearms).reset_index(drop=True)

# 4. Социально-экономические данные
socio_data = {
    'Центральный': {'плотность': 18000, 'безработица': 6.1, 'индекс_стресса': 7.2},
    'Адмиралтейский': {'плотность': 15000, 'безработица': 7.0, 'индекс_стресса': 6.8},
    'Невский': {'плотность': 12000, 'безработица': 8.3, 'индекс_стресса': 8.1},
    'Красносельский': {'плотность': 6000, 'безработица': 9.1, 'индекс_стресса': 8.5},
    'Фрунзенский': {'плотность': 9000, 'безработица': 8.7, 'индекс_стресса': 8.3},
    'Московский': {'плотность': 8500, 'безработица': 9.5, 'индекс_стресса': 8.7},
    'Приморский': {'плотность': 7000, 'безработица': 7.8, 'индекс_стресса': 7.5},
    'Выборгский': {'плотность': 5000, 'безработица': 8.0, 'индекс_стресса': 7.0},
    'Калининский': {'плотность': 11000, 'безработица': 8.9, 'индекс_стресса': 8.4},
    'Кировский': {'плотность': 7500, 'безработица': 8.2, 'индекс_стресса': 7.9},
    'Колпинский': {'плотность': 4000, 'безработица': 8.5, 'индекс_стресса': 8.6},
    'Петродворцовый': {'плотность': 3000, 'безработица': 5.5, 'индекс_стресса': 5.0},
    'Пушкинский': {'плотность': 2500, 'безработица': 6.0, 'индекс_стресса': 4.8},
    'Василеостровский': {'плотность': 14000, 'безработица': 6.5, 'индекс_стресса': 7.0},
    'Красногвардейский': {'плотность': 6500, 'безработица': 8.6, 'индекс_стресса': 8.0},
    'Кронштадтский': {'плотность': 3500, 'безработица': 6.2, 'индекс_стресса': 6.5}
}

for feature in ['плотность', 'безработица', 'индекс_стресса']:
    df[feature] = df['Район'].map(lambda x: socio_data.get(x, {}).get(feature, np.nan))

# 5. Симуляция погоды
np.random.seed(42)
df['температура'] = np.random.normal(12, 8, len(df))
df['осадки'] = np.random.choice([0, 1], len(df), p=[0.7, 0.3])

# 6. Финальный датасет
features = [
    'Час', 'День_недели', 'Месяц', 'Год', 'Время_суток',
    'Широта', 'Долгота', 'История_90дн',
    'плотность', 'безработица', 'индекс_стресса',
    'температура', 'осадки',
    'Стрельба'
]
final_df = df[features].dropna()

# 7. Обучение модели
print("🧠 Обучение модели...")
X = final_df.drop('Стрельба', axis=1)
y = final_df['Стрельба']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# Оценка
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# 8. Прогноз на ближайшие 3 часа
print("📍 Прогноз риска на ближайшие 3 часа...")
now = datetime.now(pytz.timezone('Europe/Moscow'))
current_hour = now.hour
next_hours = [(current_hour + i) % 24 for i in range(3)]
current_month = now.month

# Группировка по районам и координатам
risk_data = []
for _, row in df.dropna(subset=['Широта', 'Долгота']).iterrows():
    if row['Район'] not in socio_data:
        continue
    for hour in next_hours:
        feat = {
            'Час': hour,
            'День_недели': now.weekday(),
            'Месяц': current_month,
            'Год': now.year,
            'Время_суток': 3 if 18 <= hour < 24 else 2 if 12 <= hour < 18 else 1 if 6 <= hour < 12 else 0,
            'Широта': row['Широта'],
            'Долгота': row['Долгота'],
            'История_90дн': row['История_90дн'],
            'плотность': socio_data[row['Район']]['плотность'],
            'безработица': socio_data[row['Район']]['безработица'],
            'индекс_стресса': socio_data[row['Район']]['индекс_стресса'],
            'температура': 15,
            'осадки': 0
        }
        prob = model.predict_proba(pd.DataFrame([feat]))[0][1]
        risk_data.append([row['Широта'], row['Долгота'], prob])

# 9. Построение карты
print("🗺️ Создание карты риска...")
m = folium.Map(location=[59.9343, 30.3351], zoom_start=11, tiles='OpenStreetMap')

# Тепловая карта
HeatMap(risk_data, radius=20, blur=15, gradient={0.3: 'green', 0.6: 'orange', 1: 'red'}).add_to(m)

# Легенда
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; width: 180px; height: 100px;
         background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
  <b>Риск стрельбы</b><br>
  &nbsp; <i class="fa fa-circle" style="color:green"></i> Низкий<br>
  &nbsp; <i class="fa fa-circle" style="color:orange"></i> Средний<br>
  &nbsp; <i class="fa fa-circle" style="color:red"></i> Высокий
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Информация
folium.Marker(
    location=[59.9343, 30.3351],
    popup=f"Прогноз на {current_hour}:00–{current_hour+3}:00<br>Модель: Random Forest",
    icon=folium.Icon(color='red')
).add_to(m)

# Сохранение
m.save(risk_map_path)
print(f"✅ Карта риска сохранена: {risk_map_path}")

🚀 Загрузка данных...
⚙️ Извлечение признаков...
📊 Расчёт истории стрельбы...
🧠 Обучение модели...
              precision    recall  f1-score   support

           0       0.97      1.00      0.99     15081
           1       1.00      0.04      0.08       404

    accuracy                           0.98     15485
   macro avg       0.99      0.52      0.53     15485
weighted avg       0.98      0.98      0.96     15485

📍 Прогноз риска на ближайшие 3 часа...
🗺️ Создание карты риска...
✅ Карта риска сохранена: /content/drive/MyDrive/Г/predict/risk_map_next_3h.html
